In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv')
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/sample_submission.csv')
test_data = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv')
data.head()


In [ ]:
sample_submission.head()

In [ ]:
target_cols = data.columns[-3:]
submission_date_time = test_data['date_time']


After an evaluation of the dataset, the date_time column was found to be superflous and difficult to work with due to it being a categorical feature with high cardinality. Thus one hot encoding is not an option. To overcome such problems we decided to extrapolate the hour of the day from the datetime and treat that as the feature. The hour of the day was chosen after a rigorous process of analysis of all the other variables that can be extrapolated from the datetime. The aforementioned process is implemented in the code cell below, with the `get_hour(row)` function .

In [ ]:
import datetime
def get_hour(row):
    hour = datetime.datetime.strptime(row['date_time'], "%Y-%m-%d %X").hour
    row['date_time'] = hour
    return row



In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split


In [ ]:


def generate_predictions(target_columns):
    submission_predictions = {'date_time': submission_date_time}
    for col in target_columns:
        X =  data.apply(get_hour, axis='columns').drop(target_columns, axis = 'columns')
        y = data[col]
        test_data_final = test_data.apply(get_hour, axis='columns')
        
        model = RandomForestRegressor(n_estimators = 100, random_state = 0)
        model.fit(X, y)
        predictions = model.predict(test_data_final)
        submission_predictions[col] = predictions
    submission_finale = pd.DataFrame(submission_predictions)
    return submission_finale


In [ ]:
submission_2 = generate_predictions(target_cols)
submission_2.head()

In [ ]:
submission_2.to_csv('submission.csv', index=False)